In [1]:
import theano

get_ipython().magic(u"cd '/home/hartmank/braindecode/convvisual'")
import numpy as np
get_ipython().magic(u'load_ext autoreload')
get_ipython().magic(u'autoreload 2')
import matplotlib
matplotlib.use('Agg')
from matplotlib import pyplot as plt
from matplotlib import cm

import matplotlib.lines as mlines
import seaborn
matplotlib.rcParams['figure.figsize'] = (12.0, 3.0)
matplotlib.rcParams['font.size'] = 7
seaborn.set_style('darkgrid')

from braindecode.scripts.print_results import ResultPrinter
from braindecode.csp.print_results import CSPResultPrinter
import logging
log = logging.getLogger()
log.setLevel("DEBUG")
from braindecode.scripts.train_experiments import setup_logging
setup_logging()

from braindecode.experiments.load import load_exp_and_model
from braindecode.veganlasagne.layer_util import print_layers, recompute_bnorm_layer_statistics
import lasagne

import os
import pickle

Using gpu device 0: GeForce GTX TITAN Black (CNMeM is disabled, cuDNN 5005)


/home/hartmank/braindecode/convvisual


/home/hartmank/braindecode/vienv/local/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
n_Units = 50
n_filters = 5

#modelpath = 'models/paper/ours/cnt/deep4/243'
modelpath = 'models/paper/ours/cnt/deep4/car/43'
savepath = os.path.join(modelpath,'%dBestInputsFor%dBestFilters/'%(n_Units,n_filters))
exp, model = load_exp_and_model(os.path.join('../data/',modelpath), set_invalid_to_NaN=False)
get_ipython().magic(u'cd ../')

2016-12-19 14:26:36,915 Setting n_sample preds automatically to 119
2016-12-19 14:26:36,920 Input window length is 522
2016-12-19 14:26:37,169 Setting n_sample preds automatically to 119
2016-12-19 14:26:37,171 Input window length is 522
/home/hartmank/braindecode


In [3]:
get_ipython().magic(u"cd '/home/hartmank/braindecode/convvisual'")
tmp = np.load(os.path.join('../data/',modelpath)+'.result.pkl')

print tmp.monitor_channels['test_sample_misclass'][-5:]

/home/hartmank/braindecode/convvisual
[0.1469800420168067, 0.13912815126050415, 0.17691701680672267, 0.14325105042016806, 0.13679096638655464]


In [4]:
from braindecode.veganlasagne.layers import FinalReshapeLayer
print model
all_l = lasagne.layers.get_all_layers(model)
model = all_l[-3]
model = FinalReshapeLayer(model,remove_invalids=True,flatten=False)
model = lasagne.layers.DimshuffleLayer(model,(0,2,1,3))
model = lasagne.layers.ReshapeLayer(model,(-1,4))
model = lasagne.layers.NonlinearityLayer(model,nonlinearity=lasagne.nonlinearities.softmax)

In [5]:
print_layers(model)

 0-InputLayer                                                         [None, 128, 640, 1]
 1-DimshuffleLayer                                                    (None, 1, 640, 128)
 2-DropoutLayer             0.000                                    
 3-Conv2DLayer              10x1                                      (None, 25, 631, 128)
 4-Conv2DAllColsLayer       1x128                                     (None, 25, 631, 1)
 5-BatchNormLayer                                     elu            
 6-Pool2DLayer              3x1                       max             (None, 25, 629, 1)
 7-StrideReshapeLayer           ::3 ::1                               (None, 25, 210, 1)
 8-NonlinearityLayer                                                 
 9-DropoutLayer             0.500                                    
10-Conv2DLayer              10x1                                      (None, 50, 201, 1)
11-BatchNormLayer                                     elu            
12-Pool2DLayer         

In [6]:
get_ipython().magic(u"cd '/home/hartmank/braindecode'")
exp.dataset.load()

/home/hartmank/braindecode
2016-12-19 14:26:44,480 Load Training Set...
2016-12-19 14:27:08,550 Load Test Set...
2016-12-19 14:27:12,920 Clean Training Set...
2016-12-19 14:27:14,090 Rejected channels: []
2016-12-19 14:27:14,091 #Clean trials:     895
2016-12-19 14:27:14,092 #Rejected trials:  2
2016-12-19 14:27:14,093 Fraction Clean:    99.0%
2016-12-19 14:27:14,093 (from maxmin):     2
2016-12-19 14:27:14,094 (from var):        0
2016-12-19 14:27:14,428 Clean Test Set...
2016-12-19 14:27:14,641 Rejected channels: []
2016-12-19 14:27:14,642 #Clean trials:     160
2016-12-19 14:27:14,642 #Rejected trials:  0
2016-12-19 14:27:14,643 Fraction Clean:    100.0%
2016-12-19 14:27:14,643 (from maxmin):     0
2016-12-19 14:27:14,644 (from var):        0
2016-12-19 14:27:14,645 Create Cleaned Cnt Sets...
2016-12-19 14:27:16,795 Create sets from cleaned cnt...
2016-12-19 14:27:16,797 Preprocess continuous signal...
2016-12-19 14:27:57,463 Loaded dataset with shape: (1725659, 128, 1, 1)
2016-12-1

In [7]:
ls ..

braindecode/  data/  job.sh.e112744  job.sh.o112744


In [8]:
#print 'loaded'
datasets = exp.dataset_provider.get_train_merged_valid_test(exp.dataset)
#print 'loaded2'
#inputs,targets = datasets['test'].get_data()
exp.iterator.batch_size = 100
test_batches = list(exp.iterator.get_batches(datasets['test'], shuffle=False))
inputs,targets = test_batches[0]

targets = targets.reshape((len(inputs),-1,4))
targets = targets.mean(axis=1)

In [9]:

#from braindecode.veganlasagne.layers import create_pred_fn
#pred_fn = create_pred_fn(model)
#inputs,targets = test_batches[0]
#preds = pred_fn(inputs)
#print targets[0],preds.shape[0]
#print("Accuracy: {:.2f}%".format(100 * 
#    np.mean(np.argmax(targets, axis=1) == np.argmax(preds, axis=1))))

/home/hartmank/braindecode/vienv/local/lib/python2.7/site-packages/lasagne/layers/pool.py:266: UserWarning: DEPRECATION: the 'ds' parameter is not going to exist anymore as it is going to be replaced by the parameter 'ws'.
  mode=self.mode,
/home/hartmank/braindecode/vienv/local/lib/python2.7/site-packages/lasagne/layers/pool.py:266: UserWarning: DEPRECATION: the 'st' parameter is not going to exist anymore as it is going to be replaced by the parameter 'stride'.
  mode=self.mode,
/home/hartmank/braindecode/vienv/local/lib/python2.7/site-packages/lasagne/layers/pool.py:266: UserWarning: DEPRECATION: the 'padding' parameter is not going to exist anymore as it is going to be replaced by the parameter 'pad'.
  mode=self.mode,


[0 0 0 1] 11900
Accuracy: 51.68%


In [8]:
#from braindecode.datahandling.batch_iteration import compute_trial_start_end_samples,CntWindowTrialIterator
#trial_iterator = CntWindowTrialIterator(1, 1145, 521,
#            check_preds_smaller_trial_len=True)
#dir(exp.iterator)
#exp.iterator.input_time_length
#len(compute_trial_start_end_samples(targets)[0])

In [9]:
#test_batches = list(exp.iterator.get_batches(datasets['test'], shuffle=False))
#len(test_batches)
#len(test_batches[5][0])

In [10]:
len(np.where(np.any(datasets['test'].get_data()[1]!=0,axis=1))[0])

20000

In [11]:
len(datasets['test'].get_data()[1])

308544

In [19]:
get_ipython().magic(u"cd '/home/hartmank/braindecode/convvisual'")
from convvisual import *

from braindecode.veganlasagne.layers import create_pred_fn
model = lasagne.layers.get_all_layers(model)

figpath = os.path.join('/home/hartmank/data/convvisual/figures',savepath)
datpath = os.path.join('/home/hartmank/data/convvisual/RF_data',savepath)
print 'a',figpath
if not os.path.isdir(figpath):
    os.makedirs (figpath)
if not os.path.isdir(datpath):
    os.makedirs (datpath)

/home/hartmank/braindecode/convvisual
a /home/hartmank/data/convvisual/figures/models/paper/ours/cnt/deep4/car/43/50BestInputsFor5BestFilters/


In [21]:
layer_list = [3,5,11,17,23,28]
X_reshapes = [None,None,3,9,27,81]
splits = [1,1,4,4,8,8]

class_defs = [[1,0,0,0],
             [0,1,0,0],
             [0,0,1,0],
             [0,0,0,1]]

,
    RF_model,RF_layer = check_if_finalreshape_is_needed(model,layer)
    #Get highest input/unit activation combination
    l_RF = receptive_field_build_deconv_layers(RF_model[RF_layer],RF_model[1],
                                                   use_learned_W=False,X_reshape=X_reshape)
    pred_fn = create_pred_fn(RF_model[RF_layer])
    output = pred_fn(list(inputs))
    
    c=0
    for class_def in class_defs:
        class_indeces = np.where(np.all(np.equal(targets,class_def),axis=1))[0]
        mean_filters = output[class_indeces].mean(axis=3).mean(axis=2).mean(axis=0)
        print 'Got filter indeces'
        print mean_filters.shape,mean_filters.argmax(axis=0)
        max_filters = mean_filters.argsort(axis=0)[::-1][:n_filters]
    
        max_ind = list()
        for filter in max_filters:
            print 'Filter',filter
            max_ind.append(get_most_active_units_in_layer_from_output(output,RF_layer,filter,n_units=n_Units,abs_act=False))
        #del output
        print 'Max filter finished'
        batches = np.array_split(np.arange(len(max_ind)),split)
        
        X_RF_complete = np.array([])
        max_ind = np.asarray(max_ind)
        for batch in batches:
            print 'Batch'
            if len(batch)==0:
                break

            max_ind_ = max_ind[batch]
            max_ind_shape = max_ind_.shape
            max_ind_ = max_ind_.reshape((1,-1,max_ind_shape[2])).squeeze()

            X_RF_complete_,mask = get_receptive_field_masked_inputs(inputs,max_ind_,l_RF)

            X_RF_shape = X_RF_complete_.shape
            X_RF_complete_ = X_RF_complete_.reshape(max_ind_shape[0],-1,X_RF_shape[1],X_RF_shape[2],X_RF_shape[3])
            X_RF_complete_.shape

            X_RF_complete = np.vstack([X_RF_complete, X_RF_complete_]) if X_RF_complete.size else X_RF_complete_

        pkl = open(os.path.join(datpath,'Class%d_Layer%02d.data'%(c,layer)), 'w')
        savedata = {'RF':X_RF_complete, 'Units':max_ind, 'filters':max_filters, 'filter_act':mean_filters}
        pickle.dump(savedata,pkl)
        pkl.close()
        c+=1
        
pkl = open(os.path.join(datpath,'X_Y_Data.data'), 'w')
savedata = {'X':inputs,'Y':targets}
pickle.dump(savedata,pkl)
pkl.close()

Got filter indeces
(200,) 137
Filter 137
Filter 180
Filter 93
Filter 96
Filter 83
Max filter finished
Batch
!!!W!!! Prod{acc_dtype=int64}.0
!!!W!!! Prod{acc_dtype=int64}.0
!!!W!!! Prod{acc_dtype=int64}.0
Batch
!!!W!!! Prod{acc_dtype=int64}.0
!!!W!!! Prod{acc_dtype=int64}.0
!!!W!!! Prod{acc_dtype=int64}.0
Batch
!!!W!!! Prod{acc_dtype=int64}.0
!!!W!!! Prod{acc_dtype=int64}.0
!!!W!!! Prod{acc_dtype=int64}.0
Batch
!!!W!!! Prod{acc_dtype=int64}.0
!!!W!!! Prod{acc_dtype=int64}.0
!!!W!!! Prod{acc_dtype=int64}.0
Batch
!!!W!!! Prod{acc_dtype=int64}.0
!!!W!!! Prod{acc_dtype=int64}.0
!!!W!!! Prod{acc_dtype=int64}.0
Batch
Got filter indeces
(200,) 180
Filter 180
Filter 137
Filter 96
Filter 93
Filter 83
Max filter finished
Batch
!!!W!!! Prod{acc_dtype=int64}.0
!!!W!!! Prod{acc_dtype=int64}.0
!!!W!!! Prod{acc_dtype=int64}.0
Batch
!!!W!!! Prod{acc_dtype=int64}.0
!!!W!!! Prod{acc_dtype=int64}.0
!!!W!!! Prod{acc_dtype=int64}.0
Batch
!!!W!!! Prod{acc_dtype=int64}.0
!!!W!!! Prod{acc_dtype=int64}.0
!!!W!!

In [23]:
print max_ind.shape

(4, 50, 4)


In [24]:
print batches

[array([0]), array([1]), array([2]), array([3]), array([], dtype=int64), array([], dtype=int64), array([], dtype=int64), array([], dtype=int64)]
